= Лабораторная работа 1: TF-IDF, Word2Vec =

БВТ2202, Градов Артём Николаевич

Цель - реализовать алгоритм Word2Vec, рассчитать TF-IDF для произвольного набора данных(выбрано - статьи про хоккей).

In [2]:
pip install pymorphy3 nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import re
import nltk
import pymorphy3
import pandas as pd

from tqdm import tqdm
from nltk.stem.snowball import RussianStemmer
from sklearn.model_selection import train_test_split

import torch

from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

tqdm.pandas()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
train_dataframe = pd.read_csv("/content/drive/MyDrive/train.csv").iloc[:2000]
valid_dataframe = pd.read_csv("/content/drive/MyDrive/test.csv").iloc[:400]

In [6]:
def preproc(text):
  text = text.lower()
  text = re.sub(r"([^\w\s]\s)", r" \1", text)
  text = re.sub(r"(\d)\.", r"\1 . ", text)
  text = re.sub(r"\s+", r" ", text)
  text = text.strip()
  text = text.split(" ")
# это лемантизация. вариант со стеммингом еще дольше
  morph = pymorphy3.MorphAnalyzer()
  text = [morph.parse(word)[0].normal_form for word in text]

  return text
# получили набор токенов

def slicingWindow(sequence):
  pairs = []
  for index, word in enumerate(sequence):
    for jndex in [-2, -1, 1, 2]:
      if 0 <= index+jndex < len(sequence):
        pairs.append((sequence[index], sequence[index+jndex]))
  return pairs
# все пары слов

In [7]:
train_pairs = train_dataframe["text"].progress_apply(preproc).progress_apply(slicingWindow).explode().tolist()
valid_pairs = valid_dataframe["text"].progress_apply(preproc).progress_apply(slicingWindow).explode().tolist()

100%|██████████| 400/400 [00:00<00:00, 10377.83it/s]


In [8]:
class PairDataset(torch.utils.data.Dataset):
  def __init__(self, pairs):
    self.pairs = pairs

  def __len__(self):
    return len(self.pairs)

  def __getitem__(self, index):
    return self.pairs[index][0], self.pairs[index][1]

train_dataset = PairDataset(train_pairs)
valid_dataset = PairDataset(valid_pairs)


In [9]:
# простой токенизатор для слов
# есть два словаря - это не индексы слова в предложении, это все слова датасета и индексы для них

class Tokenizer:
  def __init__(self, set_of_words):
    self.__word2index = {} #слово в токен
    self.__index2word = {} #токен в слово
    for index, value in enumerate(set_of_words):
      self.__word2index[value] = index
      self.__index2word[index] = value

  def __call__(self, sequence):
    tokens = []
    for word in sequence:
      try:
        tokens.append(self.__word2index[word.lower()])
      except:
        raise ValueError("Такого слова нет в токенизаторе")
    return torch.tensor(tokens)

  def decode(self, sequence):
    words = []
    for index in sequence:
      try:
        words.append(self.__index2word[index])
      except:
        raise ValueError("Такого токена нет в токенизаторе")
    return words

  def __len__(self):
    return len(self.__word2index)

tokenizer_dataframe = pd.concat([train_dataframe["text"], valid_dataframe["text"]])
tokenizer = Tokenizer(set(tokenizer_dataframe.progress_apply(preproc).explode().tolist()))
tokenizer(preproc("Привет меня зовут Руслан"))

100%|██████████| 2400/2400 [03:44<00:00, 10.71it/s]


tensor([15548, 16242,  9656, 18258])

In [10]:
class word2vec(nn.Module):
  def __init__(self, num_dict, hidden):
    super().__init__()
    self.encode = nn.Embedding(num_dict, hidden)
    self.decode = nn.Linear(hidden, num_dict)

  def forward(self, x):
    out = self.encode(x)
    out = self.decode(out)
    return out

num_dict = 18728
hidden = 300
batch_size = 16

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = word2vec(num_dict, hidden).to(device)
optim = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = range(10)

train_loss = []
test_loss = []
train_acc = []
test_acc = []

cuda


In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

for epoch in epochs:
  print(f"Epoch = {epoch}")

  train_loss_mean = 0
  test_loss_mean = 0
  train_acc_mean = 0
  test_acc_mean = 0

  for data, target in (pbar := tqdm(train_dataloader)):
    optim.zero_grad()

    data = tokenizer(data).to(device)
    target = tokenizer(target).to(device)

    out = model(data)

    b_train_acc = float((out.argmax(dim=1) == target).sum()/batch_size)
    train_acc_mean += b_train_acc

    target = F.one_hot(target, num_classes=num_dict).float()
    loss = loss_fn(out, target)
    loss.backward()
    train_loss_mean += loss.item()

    optim.step()

    pbar.set_postfix({"Loss": loss.item(), "Accuracy": b_train_acc}, refresh=True)

  for data, target in (pbar := tqdm(valid_dataloader)):
    with torch.no_grad():
      data = tokenizer(data).to(device)
      target = tokenizer(target).to(device)

      out = model(data)
      b_test_acc = float((out.argmax(dim=1) == target).sum()/batch_size)
      test_acc_mean += b_test_acc

      target = F.one_hot(target, num_classes=num_dict).float()
      loss = loss_fn(out, target)
      test_loss_mean += loss.item()

      pbar.set_postfix({"Loss": loss.item(), "Accuracy": b_test_acc}, refresh=True)

  train_loss.append(train_loss_mean/len(train_dataloader))
  test_loss.append(test_loss_mean/len(valid_dataloader))
  train_acc.append(train_acc_mean/len(train_dataloader))
  test_acc.append(test_acc_mean/len(valid_dataloader))

  print(f"Train_loss: {train_loss_mean/len(train_dataloader)}")
  print(f"Valid_loss: {test_loss_mean/len(valid_dataloader)}")
  print(f"Train_acc: {train_acc_mean/len(train_dataloader)}")
  print(f"Valid_acc: {test_acc_mean/len(valid_dataloader)}")

Epoch = 0


100%|██████████| 6606/6606 [00:13<00:00, 496.46it/s, Loss=9.9, Accuracy=0.125]


Train_loss: 7.9384184085541
Valid_loss: 8.060679777261166
Train_acc: 0.045728566369402146
Valid_acc: 0.052736148955495006
Epoch = 1


100%|██████████| 6606/6606 [00:13<00:00, 495.36it/s, Loss=10.4, Accuracy=0.125]


Train_loss: 7.139791352186771
Valid_loss: 8.262530702372374
Train_acc: 0.05867285341746885
Valid_acc: 0.0542310021192855
Epoch = 2


...

Train_loss: 6.557631070183785
Valid_loss: 8.441949456740682
Train_acc: 0.06887254112874666
Valid_acc: 0.05669088707235846
Epoch = 8


100%|██████████| 6606/6606 [00:13<00:00, 506.77it/s, Loss=10.5, Accuracy=0.125]


Train_loss: 6.497442089940625
Valid_loss: 8.46160705759783
Train_acc: 0.06995911271369241
Valid_acc: 0.0565489706327581
Epoch = 9


100%|██████████| 6606/6606 [00:13<00:00, 501.31it/s, Loss=10.5, Accuracy=0.125]


Train_loss: 6.438648970127643
Valid_loss: 8.476053698078054
Train_acc: 0.07148433727181996
Valid_acc: 0.056321904329397515


TF-IDF

term frequency, inverse document frequency / количество слов в документе, количество документов с этим словом

TF = (N экземпляров конкретного слова в документе) / (N слов в документе)

IDF = log((N документов) / (N документов с этим словом))

TF-IDF = TF * IDF => вес слова в каждом документе

Так оцениваем важность слова в документе относительно всего корпуса


In [12]:
import os
import re
import numpy as np
from collections import defaultdict
path = 'rec.sport.hockey'

def findTf(path):
    tf = defaultdict(lambda: defaultdict(int))  # созд. ключи с 0 вместо ошибки
    doc_word_counts = {}  # id: N
    document_count = 0 #99

    # подсчеты
    for filename in os.listdir(path):
        doc_id = filename
        with open(os.path.join(path, filename), 'r', encoding='latin-1') as file:
            # content = file.readlines()
            content = file.read().lower()
            lines = content.splitlines()
            word_count = 0
    # плохо ли то что слова попадаются 1-3 раза? размер доков разный?
            for line in lines:
                if re.match(r">>\w|>\w|\|>\w|-\w", line):
                    words = line.split()
                    words = [word.strip('<>-') for word in words]
                    words = [word.rstrip('.,!?') for word in words]
                    words = [word.lower() for word in words]
                    word_count += len(words)
                    for word in words:
                        tf[word][doc_id] += 1

            doc_word_counts[doc_id] = word_count
            document_count += 1

    # tf
    for word, doc_counts in tf.items():
        for doc_id, count in doc_counts.items():
            tf[word][doc_id] = count / doc_word_counts[doc_id]

    return tf, document_count

def findIdf(tf, document_count):
    idf = {}
    for word in tf:
        docs_include = len(tf[word])
        idf[word] = np.log(document_count / (docs_include + 1))
    return idf

def findTfidf(tf, idf):
    tfidf = defaultdict(lambda: defaultdict(float))  # tfidf[word][doc_id]
    for word, doc_counts in tf.items():
        for doc_id, tf_value in doc_counts.items():
            tfidf[word][doc_id] = float(np.round(tf_value * idf[word], 7))
    return tfidf


tf, document_count = findTf(path)
# print(tf) #'rally': defaultdict(<class 'int'>, {'54776': 0.03225806451612903})
print(tf['when']['52550'], "\n")

idf = findIdf(tf, document_count)

tfidf = findTfidf(tf, idf)

for word, doc_counts in tfidf.items():
    if len(dict(doc_counts)) >= 3:
        print(f'TF-IDF слова "{word}": {dict(doc_counts)}')


FileNotFoundError: [Errno 2] No such file or directory: 'rec.sport.hockey'

PyTorch



### 1. class SkipGramModel(nn.Module)

`SkipGramModel` - это класс, который наследуется от `nn.Module`, базового класса для всех нейронных сетей в PyTorch. Он предоставляет структуру для создания модели обучения.

- **Определение модели**: Внутри конструктора `__init__` можно определить все компоненты модели, такие как слои, которые будут использоваться.
- **Метод forward**: Определяет, как проходить данные через модель. В данном случае он описывает, как данные обрабатываются в модели Skip-Gram, чтобы предсказать вероятности контекстных слов на основе целевых слов.

### 2. nn.Embedding(vocab_size, embedding_size)

`nn.Embedding` - это специальный слой, который используется для представления категориальных данных (например, слов) в виде плотных векторных представлений.

- **vocab_size**: Количество уникальных слов в словаре модели. Это будет размерность входного пространства.
- **embedding_size**: Размерность создаваемых векторов для каждого слова. Это влияет на общее качество представления слов и объём обучаемых параметров.

Пример: `nn.Embedding(1000, 100)` создаст матрицу весов 1000x100, где каждая строка представляет слово в виде 100-мерного вектора.

### 3. torch.matmul(in_embeds, out_embeds.t())

`torch.matmul` - это функция для выполнения матричного умножения между двумя тензорами.

- **in_embeds**: Векторные представления целевых слов, полученные из входного слоя (`in_embeddings`).
- **out_embeds.t()**: Транспонированные векторные представления всех слов, используемые в качестве выходных. Транспонирование нужно для того, чтобы размеры матриц совпадали для перемножения.

Эта операция позволяет эффективно вычислить предсказания модели, сравнивая векторы целевых слов с векторами всех контекстных слов, получая вероятностную оценку.

### 4. optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer.step()

- **optim.SGD**: Это реализация стохастического градиентного спуска (SGD) как метода оптимизации. Он обновляет параметры модели в направлении, которое минимизирует функцию потерь.
- **model.parameters()**: Метод, который возвращает все параметры (веса) модели, которые подлежат обучению.
- **lr=learning_rate**: Научная скорость, по которой оптимизатор обновляет параметры модели.
- `optimizer.step()`: Метод, который обновляет параметры модели, используя значения накапливаемых градиентов.

### 5. loss_function = nn.CrossEntropyLoss()
loss = loss_function(scores, context_tensor)
loss.backward()

- **nn.CrossEntropyLoss**: Функция потерь, которая будет использоваться для обучения модели. Она сочетает в себе softmax и потерю логарифмического распределения для многоклассовой классификации.
- **scores**: Результаты предсказания модели (логиты) для целевого слова.
- **context_tensor**: Индексы контекстных слов, которые мы пытаемся предсказать. Эта информация используется, чтобы вычислить, насколько хорошо модель предсказывает слова на основе целевого.
- `loss.backward()`: Запускает обратное распространение ошибки, вычисляя градиенты для всех обучаемых параметров модели.

### 6. target_tensor = torch.tensor([target], dtype=torch.long)
context_tensor = torch.tensor([context], dtype=torch.long)

- **torch.tensor**: Создает новый тензор из данных, переданных в виде списка или массива.
- **target_tensor**: Он содержит индекс целевого слова, которое мы берем как вход на шаге обучения. Его размерность - это 1.
- **context_tensor**: Он содержит индексы контекстных слов, которые модель пытается предсказать для данного целевого слова. Также размерность - 1.

Использование `dtype=torch.long` означает, что элементы тензора будут целыми числами, что является необходимым для индексов, так как индексы слов в `nn.Embedding` должны быть целыми.

Таким образом, каждый элемент кода выполняет важную функцию в процессе построения и обучения модели, от создания векторных представлений до управления обновлением параметров.